In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data_train_dir = 'D:/Desktop/Itwill ws/shiny_project/fingers/train/'
data_test_dir = 'D:/Desktop/Itwill ws/shiny_project/fingers/test/'

In [4]:
train_dir = 'D:/Desktop/Itwill ws/shiny_project/working/train_dir'
val_dir = 'D:/Desktop/Itwill ws/shiny_project/working/val_dir'
test_dir = 'D:/Desktop/Itwill ws/shiny_project/working/test_dir'

In [4]:
def create_directory(dir_name):
    dirs = [os.path.join(dir_name, "L"), os.path.join(dir_name, "R")]
    
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    
    for dir in dirs:
        for i in range(6):
            os.makedirs(dir + str(i))

In [5]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [6]:
paths_train = []
paths_test = []

In [7]:
count_data = {'0L': 0,
              '1L': 0,
              '2L': 0,
              '3L': 0,
              '4L': 0,
              '5L': 0,
              '0R': 0,
              '1R': 0,
              '2R': 0,
              '3R': 0,
              '4R': 0,
              '5R': 0}

In [13]:
for dirname, _, filenames in os.walk(data_train_dir):
    for filename in filenames:
        paths_train.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk(data_test_dir):
    for filename in filenames:
        paths_test.append(os.path.join(dirname, filename))

In [14]:
for filename in paths_train:
    for key in count_data.keys():
        if filename[-6:-4] == key:
            count_data[key] += 1
            
for filename in paths_test:
    for key in count_data.keys():
        if filename[-6:-4] == key:
            count_data[key] += 1

In [15]:
counts = count_data.values()
groups = count_data.keys()

In [16]:
def copy_images(start_index, end_index, paths, dest_dir):
    for i in range(start_index, end_index):
        dest_path = os.path.join(dest_dir, paths[i][-5] + paths[i][-6])
        shutil.copy2(paths[i], dest_path)

In [17]:
# Part of the validation data set
validation_data_proportion = 0.15

In [18]:
# copying images from input directory to output
copy_images(0, len(paths_train) - int(validation_data_proportion * len(paths_train)),
            paths_train, train_dir)
copy_images(len(paths_train) - int(validation_data_proportion * len(paths_train)),
            len(paths_train), paths_train, val_dir)
copy_images(0, len(paths_test), paths_test, test_dir)

In [19]:
target_size = (128, 128)
batch_size = 50
mode = 'categorical'

In [20]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [21]:
train_gen = datagen.flow_from_directory(train_dir,
                                       target_size=target_size,
                                       batch_size=batch_size,
                                       class_mode=mode)

Found 18000 images belonging to 12 classes.


In [22]:
val_gen = datagen.flow_from_directory(val_dir,
                                       target_size=target_size,
                                       batch_size=batch_size,
                                       class_mode=mode)

Found 5400 images belonging to 12 classes.


In [23]:
test_gen = datagen.flow_from_directory(test_dir,
                                       target_size=target_size,
                                       batch_size=batch_size,
                                       class_mode=mode)

Found 3600 images belonging to 12 classes.


In [24]:
train_gen.class_indices

{'L0': 0,
 'L1': 1,
 'L2': 2,
 'L3': 3,
 'L4': 4,
 'L5': 5,
 'R0': 6,
 'R1': 7,
 'R2': 8,
 'R3': 9,
 'R4': 10,
 'R5': 11}

In [25]:
len_train_data = len(train_gen.filenames)
len_test_data = len(test_gen.filenames)
len_val_data = len(val_gen.filenames)

In [26]:
best_model_path = 'best_model.h5'
checkpoint_callback = ModelCheckpoint(best_model_path,
                                     monitor='val_accuracy',
                                     save_best_only=True,
                                     verbose=1)
reduce_callback = ReduceLROnPlateau(monitor='val_accuracy',
                                   patience=3,
                                   factor=0.5,
                                   min_lr=0.00001,
                                   verbose=1)
callbacks_list = [checkpoint_callback, reduce_callback]

In [27]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(12, activation='softmax'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 128)       0

In [29]:
model.compile(loss='categorical_crossentropy',
             optimizer='RMSprop',
             metrics=['accuracy', 'AUC'])

In [1]:
train_gen.shape

NameError: name 'train_gen' is not defined

In [30]:
history = model.fit_generator(train_gen,
                   steps_per_epoch=len_train_data // batch_size,
                   epochs=3,
                   validation_data=val_gen,
                   validation_steps=len_val_data // batch_size,
                   verbose=1,
                   callbacks=callbacks_list)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 360 steps, validate for 108 steps
Epoch 1/3
359/360 [============================>.] - ETA: 0s - loss: 0.3238 - accuracy: 0.9336 - AUC: 0.9928
Epoch 00001: val_accuracy improved from -inf to 0.99926, saving model to best_model.h5
360/360 [==============================] - 82s 227ms/step - loss: 0.3229 - accuracy: 0.9338 - AUC: 0.9928 - val_loss: 0.0021 - val_accuracy: 0.9993 - val_AUC: 1.0000
Epoch 2/3
359/360 [============================>.] - ETA: 0s - loss: 0.0479 - accuracy: 0.9962 - AUC: 0.9990
Epoch 00002: val_accuracy improved from 0.99926 to 0.99981, saving model to best_model.h5
360/360 [==============================] - 71s 198ms/step - loss: 0.0477 - accuracy: 0.9962 - AUC: 0.9990 - val_loss: 7.7553e-04 - val_accuracy: 0.9998 - val_AUC: 1.0000
Epoch 3/3
359/360 [============================>.] - ETA: 0s - loss: 0.0129 - accuracy: 0.9988 - AU

In [31]:
model.load_weights(best_model_path)

In [32]:
testing_model = model.evaluate_generator(test_gen, len_test_data // batch_size, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']
72/72 [==============================] - 4s 56ms/step - loss: 2.4114e-04 - accuracy: 0.9997 - AUC: 1.0000


In [33]:
print('Percentage of correct responses: ' + str(int(testing_model[1] * 10000) / 100) + '%')

Percentage of correct responses: 99.97%


In [41]:
import gc
tf.compat.v1.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()

166

In [ ]:
import tensorflow as tf 
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
session = tf.Session(config=config)

In [37]:
# status initialize

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


# out of memory
import tensorflow as tf
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(allow_growth=True)

AttributeError: module 'tensorflow' has no attribute 'GPUOptions'

In [ ]:
#from keras.datasets import cifar10
#from keras.models import Sequential, save_model
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D
#import numpy as np
#from keras.utils import np_utils
#from keras.layers.normalization import BatchNormalization


# 필요한 모듈을 임폴트 한다. 
import loader3
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation

# 하이퍼 파라미터를 설정한다. 
batch_size = 28
num_classes = 2
epochs = 100
 
# 이미지 데이터와 라벨위치 
train_image = 'D:\\data\\leafs\\images\\train_resize\\'
test_image = 'D:\\data\\leafs\\images\\test_resize\\'
train_label = 'D:\\data\\leafs\\images\\train_label.csv'
test_label = 'D:\\data\\leafs\\images\\test_label.csv'

# 데이터 로드 
x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)
x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)      

print ( loader3.image_load(train_image).shape )
print ( loader3.image_load(test_image).shape )
print ( loader3.label_load(train_label).shape)
print ( loader3.label_load(test_label).shape )

#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# One hot Encoding
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

# 모델 구성 
model = Sequential()
# 첫번째 층은 반드시 input_shape 를 지정해야한다. 
#  x_train.shape ( 19000, 32, 32, 3 )   
# input_shape=x_train.shape[1:] (32, 32, 3) 
# mnist 와 비교하면 mnist 의 경우는 첫층이 완전연결계층이므로 
# Flatten 시켜서  Flatten(input_shape=(28,28) ) 이렇게 입력했었고
# Convolution 층은 이미지의 형상이 무시되지 않도록 입력해야하기 때문에 
# (32, 32, 3)  이렇게 3차원으로 넣어야한다. 

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
# 모델 설정 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# 데이터 정규화 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
# 모델 학습 
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)
 
# 모델 평가 
scores = model.evaluate(x_test, y_test, verbose=0)

#[0.3352026641368866, 0.9929999709129333]
#    오차                        정확도 

print("CNN Error: %.2f%%" % (100-scores[1]*100))
 
save_model(model, "D:\\data\\leafs\\leaf.h5")


